<a href="https://colab.research.google.com/github/IshtiSikder/Relationship-extraction-from-texts/blob/main/RelationshipExtractor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



# Kindly select Runtime > Run all from above to run all the following cells successively, and please read the numbered text cells with the code cells below for detailed descriptions about how the code works. 



# 1. Let's first install some necessary modules/dependencies to build our relation extraction pipeline with python

In [ ]:
pip install spacy==2.1.0

     |████████████████████████████████| 27.7 MB 37.9 MB/s 
     |████████████████████████████████| 2.1 MB 29.6 MB/s 
     |████████████████████████████████| 3.2 MB 41.0 MB/s 
     |████████████████████████████████| 82 kB 191 kB/s 
  Attempting uninstall: preshed
    Found existing installation: preshed 3.0.6
    Uninstalling preshed-3.0.6:
      Successfully uninstalled preshed-3.0.6
  Attempting uninstall: plac
    Found existing installation: plac 1.1.3
    Uninstalling plac-1.1.3:
      Successfully uninstalled plac-1.1.3
  Attempting uninstall: blis
    Found existing installation: blis 0.4.1
    Uninstalling blis-0.4.1:
      Successfully uninstalled blis-0.4.1
  Attempting uninstall: thinc
    Found existing installation: thinc 7.4.0
    Uninstalling thinc-7.4.0:
      Successfully uninstalled thinc-7.4.0
  Attempting uninstall: spacy
    Found existing installation: spacy 2.2.4
    Uninstalling spacy-2.2.4:
      Successfully uninstalled spacy-2.2.4
ERROR: pip's dependency resol

In [ ]:
!python -m spacy download en_core_web_sm

     |████████████████████████████████| 11.1 MB 6.6 MB/s 
  Created wheel for en-core-web-sm: filename=en_core_web_sm-2.1.0-py3-none-any.whl size=11074431 sha256=18a09e97dc72e660fe1259d8035f86f2df8366c9c2700d9878f33a28417070a3
  Stored in directory: /tmp/pip-ephem-wheel-cache-1f1q5a7_/wheels/59/4f/8c/0dbaab09a776d1fa3740e9465078bfd903cc22f3985382b496
Successfully built en-core-web-sm
  Attempting uninstall: en-core-web-sm
    Found existing installation: en-core-web-sm 2.2.5
    Uninstalling en-core-web-sm-2.2.5:
      Successfully uninstalled en-core-web-sm-2.2.5
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')


In [ ]:
pip install neuralcoref

     |████████████████████████████████| 286 kB 5.2 MB/s 
     |████████████████████████████████| 131 kB 46.9 MB/s 
     |████████████████████████████████| 79 kB 7.2 MB/s 
     |████████████████████████████████| 8.5 MB 47.5 MB/s 
     |████████████████████████████████| 127 kB 47.9 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


In [ ]:
pip install folium==0.2.1

     |████████████████████████████████| 69 kB 3.7 MB/s 
  Created wheel for folium: filename=folium-0.2.1-py3-none-any.whl size=79809 sha256=0adf8133b20cf5ea37aaa2b4f73d357b22847c7997ce23a3a28f8847f3b25a1d
  Stored in directory: /root/.cache/pip/wheels/9a/f0/3a/3f79a6914ff5affaf50cabad60c9f4d565283283c97f0bdccf
Successfully built folium
  Attempting uninstall: folium
    Found existing installation: folium 0.8.3
    Uninstalling folium-0.8.3:
      Successfully uninstalled folium-0.8.3


# 2. Let's now build a **coreference resolution function** using two python modules designated for the task, which are **spacy** and **neuralcoref**. neuralcoref is a production-ready neural network that we will run on one of spacy's annotated english language corpuses (**en_core_web_sm**) to conduct coreference resolution on any given text.

# **Coreference resolution** denotes to the task of teaching a machine how to link all the pronouns used in a sentence with related nouns. By establishing the coreferences, the pipeline avoids disambiguation among entities. 

In [ ]:
import spacy
import neuralcoref

nlp = spacy.load('en_core_web_sm')
neuralcoref.add_to_pipe(nlp)

def coref_resolution(text):
    doc = nlp(text)
    tok_list = list(token.text_with_ws for token in doc)
    for cluster in doc._.coref_clusters:
        cluster_main_words = set(cluster.main.text.split(' '))
        for coref in cluster:
            if coref != cluster.main:  
                if coref.text != cluster.main.text and bool(set(coref.text.split(' ')).intersection(cluster_main_words)) == False:
                    tok_list[coref.start] = cluster.main.text + \
                        doc[coref.end-1].whitespace_
                    for i in range(coref.start+1, coref.end):
                        tok_list[i] = ""

    return "".join(tok_list)

100%|██████████| 40155833/40155833 [00:01<00:00, 32894752.50B/s]


# 3. We will test our coreference resolution function on an example text. The text is as follows, collected from Wikipedia at random.

In [ ]:
test = "Elon Musk is a business magnate, industrial designer, and engineer. He is the founder of SpaceX. He is also early investor, CEO, and product architect of Tesla, Inc. He is also the founder of The Boring Company and the co-founder of Neuralink. Musk was born to a Canadian mother and South African father and raised in Pretoria, South Africa. He briefly attended the University of Pretoria before moving to Canada aged 17 to attend Queen's University. He transferred to the University of Pennsylvania two years later, where he received dual bachelor's degrees in economics and physics. He moved to California in 1995 to attend Stanford University, but decided instead to pursue a business career. He went on co-founding a web software company Zip2 with his brother Kimbal Musk."

In [ ]:
test

"Elon Musk is a business magnate, industrial designer, and engineer. He is the founder of SpaceX. He is also early investor, CEO, and product architect of Tesla, Inc. He is also the founder of The Boring Company and the co-founder of Neuralink. Musk was born to a Canadian mother and South African father and raised in Pretoria, South Africa. He briefly attended the University of Pretoria before moving to Canada aged 17 to attend Queen's University. He transferred to the University of Pennsylvania two years later, where he received dual bachelor's degrees in economics and physics. He moved to California in 1995 to attend Stanford University, but decided instead to pursue a business career. He went on co-founding a web software company Zip2 with his brother Kimbal Musk."

In [ ]:
results = coref_resolution(test)

# 4. If we compare the original text with the results below, we can see that the coreference resolution function properly replaced all the pronouns in the original text with corresponding nouns. 

In [ ]:
results

"Elon Musk is a business magnate, industrial designer, and engineer. Elon Musk is the founder of SpaceX. Elon Musk is also early investor, CEO, and product architect of Tesla, Inc. Elon Musk is also the founder of The Boring Company and the co-founder of Neuralink. Musk was born to a Canadian mother and South African father and raised in Pretoria, South Africa. Elon Musk briefly attended the University of Pretoria before moving to Canada aged 17 to attend Queen's University. Elon Musk transferred to the University of Pennsylvania two years later, where Elon Musk received dual bachelor's degrees in economics and physics. Elon Musk moved to California in 1995 to attend Stanford University, but decided instead to pursue a business career. Elon Musk went on co-founding a web software company Zip2 with Elon Musk brother Kimbal Musk."

# 5. We will now build a **wikifier** function that can conduct **Named Entity Linking** , which referes to the task of labelling entities in a text based on their general classification (i.e Elon Musk can be labelled as *Person*,Tesla can be labelled as *Organization* etc.). For building the wikifier function, we will use the **wikifier API**. Wikifier is a large database of labelled entities collected from Wikipedia texts. 

# Considering our text chunk, we will use the wikifier API to cross check their database for three specific labels: Person, Organization and Location. 

In [ ]:
import urllib
from string import punctuation
import nltk
import json

ENTITY_TYPES = ["human", "person", "company", "enterprise", "business", "geographic region",
                "human settlement", "geographic entity", "territorial entity type", "organization"]

def wikifier(text, lang="en", threshold=0.8):
    data = urllib.parse.urlencode([
        ("text", text), ("lang", lang),
        ("userKey", "tgbdmkpmkluegqfbawcwjywieevmza"),
        ("pageRankSqThreshold", "%g" %
         threshold), ("applyPageRankSqThreshold", "true"),
        ("nTopDfValuesToIgnore", "100"), ("nWordsToIgnoreFromList", "100"),
        ("wikiDataClasses", "true"), ("wikiDataClassIds", "false"),
        ("support", "true"), ("ranges", "false"), ("minLinkFrequency", "2"),
        ("includeCosines", "false"), ("maxMentionEntropy", "3")
    ])
    url = "http://www.wikifier.org/annotate-article"
    req = urllib.request.Request(url, data=data.encode("utf8"), method="POST")
    with urllib.request.urlopen(req, timeout=60) as f:
        response = f.read()
        response = json.loads(response.decode("utf8"))
    results = list()
    for annotation in response["annotations"]:
        if ('wikiDataClasses' in annotation) and (any([el['enLabel'] in ENTITY_TYPES for el in annotation['wikiDataClasses']])):
            if any([el['enLabel'] in ["human", "person"] for el in annotation['wikiDataClasses']]):
                label = 'Person'
            elif any([el['enLabel'] in ["company", "enterprise", "business", "organization"] for el in annotation['wikiDataClasses']]):
                label = 'Organization'
            elif any([el['enLabel'] in ["geographic region", "human settlement", "geographic entity", "territorial entity type"] for el in annotation['wikiDataClasses']]):
                label = 'Location'
            else:
                label = None

            results.append({'title': annotation['title'], 'wikiId': annotation['wikiDataItemId'], 'label': label,
                            'characters': [(el['chFrom'], el['chTo']) for el in annotation['support']]})
    return results

# 6. Let's run the cell below to check if the wikifier function is working properly. 

#We can see that some labels are wrong. For example, South Africa has been labelled as an **organization** whereas it should have been labelled as **location**. 

# To improve on this, an alternate and more robust module called **Facebook BLINK model** can be used for Named Entity Linking instead of wikifier, which is pretty dated. In this case, Facebook BLINK model hasn't been implemented because of it's extensive memory requirements, which can put a strain on the overall performance of this pipeline given the memory constraints of Google colab. 

In [ ]:
import pandas as pd
data = wikifier(results)

pd.DataFrame(data)

,title,wikiId,label,characters
0,Elon Musk,Q317521,Person,"[(0, 8), (5, 8), (68, 76), (73, 76), (104, 112..."
1,SpaceX,Q193701,Organization,"[(96, 101)]"
2,"Tesla, Inc.",Q478214,Organization,"[(168, 172), (168, 177), (168, 178)]"
3,The Boring Company,Q28874479,Organization,"[(213, 230)]"
4,South Africa,Q258,Organization,"[(304, 316), (349, 360), (349, 361)]"
5,Pretoria,Q3926,Organization,"[(339, 346), (339, 360), (408, 415)]"
6,University of Pretoria,Q604444,Organization,"[(394, 415)]"
7,University of Pennsylvania,Q49117,Organization,"[(504, 533), (508, 533), (522, 533)]"
8,Stanford University,Q41506,Organization,"[(675, 693)]"
9,Zip2,Q28222602,Organization,"[(798, 801)]"


# 7. We will now clone an open source python module named OpenNRE from Github which can build us our relation extraction model using BERT.

In [ ]:
!git clone https://github.com/thunlp/OpenNRE.git

Cloning into 'OpenNRE'...
remote: Enumerating objects: 1514, done.
remote: Counting objects: 100% (137/137), done.
remote: Compressing objects: 100% (95/95), done.
remote: Total 1514 (delta 81), reused 82 (delta 42), pack-reused 1377
Receiving objects: 100% (1514/1514), 266.83 MiB | 28.97 MiB/s, done.
Resolving deltas: 100% (901/901), done.


# 8. Changing Google colab directory to fetch requirements.txt file imported from the github repo of OpenNRE. We will use this .txt file to install necessary modules/dependencies to build the relation extraction model

In [ ]:
cd /content/OpenNRE

/content/OpenNRE


In [ ]:
!python -m pip install -r requirements.txt
!pip install torch==1.10.0
!pip install scikit-learn==0.24

     |████████████████████████████████| 748.8 MB 19 kB/s 
     |████████████████████████████████| 1.3 MB 38.7 MB/s 
     |████████████████████████████████| 234 kB 51.9 MB/s 
     |████████████████████████████████| 7.0 MB 23.1 MB/s 
     |████████████████████████████████| 1.5 MB 36.9 MB/s 
     |████████████████████████████████| 2.9 MB 44.6 MB/s 
     |████████████████████████████████| 895 kB 44.7 MB/s 
     |████████████████████████████████| 1.2 MB 546 kB/s 
     |████████████████████████████████| 749 kB 48.3 MB/s 
  Attempting uninstall: regex
    Found existing installation: regex 2019.12.20
    Uninstalling regex-2019.12.20:
      Successfully uninstalled regex-2019.12.20
  Attempting uninstall: pluggy
    Found existing installation: pluggy 0.7.1
    Uninstalling pluggy-0.7.1:
      Successfully uninstalled pluggy-0.7.1
  Attempting uninstall: torch
    Found existing installation: torch 1.10.0+cu111
    Uninstalling torch-1.10.0+cu111:
      Successfully uninstalled torch-1.10.0+c

     |████████████████████████████████| 22.3 MB 764 kB/s 
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 0.22.1
    Uninstalling scikit-learn-0.22.1:
      Successfully uninstalled scikit-learn-0.22.1


# 9. We can now build the relation extraction model. We will use a pre-trained BERT model from OpenNRE called wiki80_bert_softmax, which can infer 80 different relation types.

In [ ]:
import opennre
relation_model = opennre.get_model('wiki80_bert_softmax')
relation_model = relation_model.cuda()

2022-01-09 13:47:28,076 - root - INFO - Loading BERT pre-trained checkpoint.


# 10. Finally, we will feed our original text to the relation model one sentence at a time. The sentence will first be stripped of all punctuations for ease of analysis, before it's processed by the wikifier function to extract all it's entities. For sentences with 2 or more discernible entities,  permutations of said entities will be sent through the relation model to calculate possible relationships between any 2 entities and confidence levels of said relationships. We will choose the relationship with the highest confidence interval and assign it to the entity pair in question.

In [ ]:
import itertools
import string
relations_list = []
for sentence in results.split("."):
  sentence = sentence.translate(str.maketrans('', '', string.punctuation))
  entities = wikifier(sentence)
  print('entities:',entities)
  for permutation in itertools.permutations(entities, 2):
    dic = {}
    for source in permutation[0]['characters']:
      for target in permutation[1]['characters']:
        data = relation_model.infer({'text': sentence, 'h': {'pos': [source[0], source[1] + 1]}, 't': {'pos': [target[0], target[1] + 1]}})
        dic[data[0]] = data[1]
    relations_list.append({'source': permutation[0]['title'], 'target': permutation[1]['title'], 'type': max(dic)})

entities: [{'title': 'Elon Musk', 'wikiId': 'Q317521', 'label': 'Person', 'characters': [(0, 8), (5, 8)]}]
entities: [{'title': 'Elon Musk', 'wikiId': 'Q317521', 'label': 'Person', 'characters': [(1, 9), (6, 9)]}]
entities: [{'title': 'Elon Musk', 'wikiId': 'Q317521', 'label': 'Person', 'characters': [(1, 9), (6, 9)]}, {'title': 'Tesla, Inc.', 'wikiId': 'Q478214', 'label': 'Organization', 'characters': [(63, 67), (63, 71)]}]
entities: [{'title': 'Elon Musk', 'wikiId': 'Q317521', 'label': 'Person', 'characters': [(1, 9), (6, 9)]}, {'title': 'The Boring Company', 'wikiId': 'Q28874479', 'label': 'Organization', 'characters': [(34, 51)]}, {'title': 'Neuralink', 'wikiId': 'Q29043471', 'label': 'Organization', 'characters': [(74, 82)]}]
entities: [{'title': 'Pretoria', 'wikiId': 'Q3926', 'label': 'Organization', 'characters': [(75, 82)]}]
entities: [{'title': 'Elon Musk', 'wikiId': 'Q317521', 'label': 'Person', 'characters': [(1, 9), (6, 9)]}, {'title': 'University of Pretoria', 'wikiId': 'Q

# 11. Let's now check the final results by running the cell below. The "type" column defines the relationship between same row words in the "source" and "target" columns.

# We can see that while most of the defined relationships are apparently logical, there are some mislabellings. i.e Stanford University is not Elon Musk's residence as suggested by rows 16 and 17. Also, One entity seen during only Named Entity Linking, which is Zip2, is missing while doing Named Entity Linking for Relation Extraction. These currently represent some unreliability/limitations for the pipeline and can be fixed by **joint extraction**. In joint extraction, Named Entity Linking and Relation Extraction are conducted simultaneously as opposed to consecutively like we did in this implementation.Joint extraction can be done in various ways (i.e through Integer Programming) but would require us to vastly change the current design of our pipeline.

In [ ]:
pd.DataFrame(relations_list)

,source,target,type
0,Elon Musk,"Tesla, Inc.",owned by
1,"Tesla, Inc.",Elon Musk,owned by
2,Elon Musk,The Boring Company,owned by
3,Elon Musk,Neuralink,owned by
4,The Boring Company,Elon Musk,owned by
5,The Boring Company,Neuralink,subsidiary
6,Neuralink,Elon Musk,owned by
7,Neuralink,The Boring Company,subsidiary
8,Elon Musk,University of Pretoria,work location
9,Elon Musk,Pretoria,work location
